In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from flame_net.FourierOp_Nd import FourierOp_Nd
from flame_net.DeepONet_1d import DeepONet_1d
from flame_net.ConvPDE_Nd import ConvPDE_Nd
from flame_net.lib_uti import Cdata_sys, count_learnable_params, lib_Model,lib_DataGen,LpLoss,lib_ModelTrain
from timeit import default_timer

#-----------------------
device = torch.device('cuda') # if torch.cuda.is_available() else 'cpu')


#-----------------------
nDIM = 1
data_sys = Cdata_sys('siva',[0.07],1)
#data_sys = Cdata_sys('cfd',['L1536_rho8'])
params = lib_Model.set_default_params(data_sys,nDIM)

params['T_in' ] = 1
params['T_out'] = 1
params['data_channel'] = 1
#params['data:yB_estimate']=np.array([-0.7, 1.3]) * np.pi
#params['data:yB_estimate']=np.array([-0.8, 2.4]) * np.pi
#params['data:yB_estimate']=np.array([-0.7, 2.5]) * np.pi
#params['data:AspectRatio_set'] = 1 # (2.5+0.7)/2.
params['data:nStep']  = 1
params['data:nStepSkip']=1
params['Nx']            =  128 #512 
#-----
params['train:batch_size'] = 1000
params['train:learning_rate'] = 0.0025
#-----------
#model_name = 'fourier'
#params['fourier:modes_fourier' ] = [64,64]
#params['fourier:width' ] = 20
#params['fourier:depth' ] = 4
#model_name = 'conv'
#params['conv:en1_channels' ] = [16,32,64,64,128,128,256,256]
model_name = 'onet'
#params['onet:type_branch']='conv'
params['onet:type_branch']='fc'
params['onet:P'] = 50
params['onet:trunk_featurepair']=5
params['onet:type_trunk']='simple'
#---------



train_disp, test_disp, train_PDEpara, test_PDEpara = lib_DataGen.DataGen(data_sys,params)


----- params for DataGen -----
nDIM,T_in,T_out,Nx,nStep,nStepSkip,data_channel, data_sys.sysname, data_sys.list_nu, data_sys.list_cfdfilename
------------------------------
N= 128 ,dt_Output= 0.015 ,nu_list = [0.07] ,nu_extention_list = [0.07, 0.125, 0.4, 0.7] ,NumOutPut_dt= [10 10 10 10]
ActiveNoise_Amplitude=0.000000,stepfeq=1,k = 0 
[(200, 501, 0.07, 'rand_FFT_2_8'), (200, 501, 0.07, 'rand_simple')]
num_traj= 200  leq_seq= 501  nu= 0.07   rand_FFT_2_8
Sucess: load ./data/dsol_multraj200L501_nu07_N128_dt015_rand_FFT_2_8.pkl
num_traj= 200  leq_seq= 501  nu= 0.07   rand_simple
Sucess: load ./data/dsol_multraj200L501_nu07_N128_dt015_rand_simple.pkl
[(40, 501, 0.07, 'rand_FFT_2_8'), (40, 501, 0.07, 'rand_simple')]
num_traj= 40  leq_seq= 501  nu= 0.07   rand_FFT_2_8
Sucess: load ./data/dsol_multraj40L501_nu07_N128_dt015_rand_FFT_2_8.pkl
num_traj= 40  leq_seq= 501  nu= 0.07   rand_simple
Sucess: load ./data/dsol_multraj40L501_nu07_N128_dt015_rand_simple.pkl
Reorg_xsol:num__split_seq_pierce

In [2]:
#%matplotlib inline
#plt.figure(figsize=[4,4])
#plt.contourf(  train_disp[-1,:,:,-1]  )


In [3]:
model_name_detail = lib_Model.get_model_name_detail(model_name,data_sys,params)
model = lib_Model.build_model(model_name_detail,params)

ONet_nu07_branchfc_P50_feature5
count_learnable_params= 74601


In [5]:
params['train:batch_size'] =1500
lib_ModelTrain.Train(train_disp, test_disp,train_PDEpara,test_PDEpara,model,model_name_detail,device,params )


batch_size= 1500
ntrain= 199200  ,ntest= 39840
train_l2_indentity= tensor(0.0156) ,  test_l2_indentity= tensor(0.0157)
.....................................................................................................................................
ep t[s] trainErr_norm testErr_norm train_l2 test_l2 train_l2_step test_l2_step train_rel_ind test_rel_ind 
0, 1.75, 1.000, 0.627, 0.190, 0.119, 0.190, 0.119, 12.124, 7.562
.....................................................................................................................................
1, 1.73, 0.653, 0.591, 0.124, 0.112, 0.124, 0.112, 7.918, 7.131
.....................................................................................................................................
2, 1.70, 0.599, 0.546, 0.114, 0.103, 0.114, 0.103, 7.263, 6.582
.....................................................................................................................................
3, 1.74, 0.555, 0.488, 0.105, 0.092, 0.105, 0

.....................................................................................................................................
40, 1.98, 0.301, 0.300, 0.057, 0.057, 0.057, 0.057, 3.649, 3.614
.....................................................................................................................................
41, 1.95, 0.289, 0.241, 0.055, 0.046, 0.055, 0.046, 3.509, 2.908
.....................................................................................................................................
42, 1.99, 0.291, 0.264, 0.055, 0.050, 0.055, 0.050, 3.529, 3.178
.....................................................................................................................................
43, 1.75, 0.296, 0.337, 0.056, 0.064, 0.056, 0.064, 3.588, 4.065
.....................................................................................................................................
44, 1.71, 0.297, 0.288, 0.056, 0.055, 0.056, 0.055, 3.597, 3.473
.....

.....................................................................................................................................
81, 1.71, 0.245, 0.216, 0.046, 0.041, 0.046, 0.041, 2.972, 2.609
.....................................................................................................................................
82, 1.73, 0.248, 0.203, 0.047, 0.039, 0.047, 0.039, 3.008, 2.454
.....................................................................................................................................
83, 1.74, 0.243, 0.227, 0.046, 0.043, 0.046, 0.043, 2.947, 2.738
.....................................................................................................................................
84, 1.70, 0.257, 0.226, 0.049, 0.043, 0.049, 0.043, 3.112, 2.725
.....................................................................................................................................
85, 1.73, 0.246, 0.230, 0.047, 0.044, 0.047, 0.044, 2.987, 2.774
.....

.....................................................................................................................................
122, 1.73, 0.172, 0.156, 0.033, 0.029, 0.033, 0.029, 2.089, 1.876
.....................................................................................................................................
123, 1.73, 0.173, 0.153, 0.033, 0.029, 0.033, 0.029, 2.102, 1.850
.....................................................................................................................................
124, 1.69, 0.178, 0.159, 0.034, 0.030, 0.034, 0.030, 2.156, 1.920
.....................................................................................................................................
125, 1.73, 0.173, 0.158, 0.033, 0.030, 0.033, 0.030, 2.101, 1.911
.....................................................................................................................................
126, 1.69, 0.176, 0.168, 0.033, 0.032, 0.033, 0.032, 2.133, 2.029


.....................................................................................................................................
163, 1.72, 0.169, 0.146, 0.032, 0.028, 0.032, 0.028, 2.047, 1.758
.....................................................................................................................................
164, 1.68, 0.164, 0.148, 0.031, 0.028, 0.031, 0.028, 1.986, 1.790
.....................................................................................................................................
165, 1.68, 0.166, 0.153, 0.031, 0.029, 0.031, 0.029, 2.014, 1.844
.....................................................................................................................................
166, 1.72, 0.163, 0.147, 0.031, 0.028, 0.031, 0.028, 1.980, 1.767
.....................................................................................................................................
167, 1.76, 0.174, 0.158, 0.033, 0.030, 0.033, 0.030, 2.107, 1.911


...

.....................................................................................................................................
819, 1.71, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.251, 1.275
.....................................................................................................................................
820, 1.67, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.249, 1.276
.....................................................................................................................................
821, 1.71, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.248, 1.277
.....................................................................................................................................
822, 1.73, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.249, 1.276
.....................................................................................................................................
823, 1.71, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.249, 1.275


.....................................................................................................................................
860, 1.68, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.248, 1.276
.....................................................................................................................................
861, 1.72, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.249, 1.274
.....................................................................................................................................
862, 1.67, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.248, 1.276
.....................................................................................................................................
863, 1.67, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.249, 1.277
.....................................................................................................................................
864, 1.72, 0.103, 0.106, 0.020, 0.020, 0.020, 0.020, 1.249, 1.275


.....................................................................................................................................
901, 1.98, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.245, 1.272
.....................................................................................................................................
902, 1.95, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.244, 1.272
.....................................................................................................................................
903, 1.79, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.245, 1.272
.....................................................................................................................................
904, 1.71, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.244, 1.271
.....................................................................................................................................
905, 1.67, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.245, 1.271


.....................................................................................................................................
942, 1.75, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.244, 1.272
.....................................................................................................................................
943, 1.72, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.244, 1.271
.....................................................................................................................................
944, 1.72, 0.103, 0.106, 0.019, 0.020, 0.019, 0.020, 1.245, 1.273
.....................................................................................................................................
945, 1.76, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.245, 1.271
.....................................................................................................................................
946, 1.71, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.244, 1.272


.....................................................................................................................................
983, 1.72, 0.103, 0.106, 0.019, 0.020, 0.019, 0.020, 1.244, 1.274
.....................................................................................................................................
984, 1.69, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.244, 1.271
.....................................................................................................................................
985, 1.72, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.243, 1.270
.....................................................................................................................................
986, 1.70, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.243, 1.271
.....................................................................................................................................
987, 1.74, 0.103, 0.105, 0.019, 0.020, 0.019, 0.020, 1.243, 1.270
